# Week 12 Handson - Assignment/Project 2: Regression #02

As the previous assignment, the objective of this project is to explore your capability as data analyst. You are challenged to implement data mining methods, given a raw dataset. Then, you need to propose several regression methods for the data, analyze and evaluate it. 

Basic tasks you need to perform in this project: (Note: you can add additional tasks that you think necessary for this project)

1.	Build regression models to predict the price for second hand cars. This step should include EDA, pre-processing, creating model and evaluation. You may use several regression methods, do benchmarking, employ additional techniques, such as ensemble method, to improve the prediction accuracy, etc.
2.	Please use the most appropriate evaluation metrics for this regression project.

Deadline: 17 November 2020, 11:59 a.m. 
 
Deliverables: 
1. Jupyter notebook (your code)
2.	Presentation <br>
**Nb**: In the end of slide and report, please write members’ names and their own contribution during this project.

In [50]:
import numpy as np
import matplotlib.pyplot as plt

## Read dataset

In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dsdmw12/data.csv


In [3]:
import pandas as pd

df = pd.read_csv('/kaggle/input/dsdmw12/data.csv')
print('Shape:', df.shape)
df.head()

# credit to prev. year DSDM students: Felix Septianus Darmawan, Maulana Akmal, Steven Sukma limanus, Ricky Kennedy

Shape: (36575, 20)


,Tahun,Kapasitas mesin,Warna,Tipe bodi,Varian,STATE,Merek,Transmisi,Model,Fitur tambahan,Nama Bursa Mobil,Tipe bahan bakar,Tipe Penjual,CITY,COUNTRY,Jarak tempuh,phone,Sistem Penggerak,price,NEIGHBOURHOOD
0,2008,>1.500 - 2.000 cc,Abu-abu,Pick-up,G Luxury,Jawa Barat,Toyota,Automatic,Innova,"['Sensor Parkir', 'Electric Window']",NaN,Bensin,Individu,Bandung Kota,Indonesia,120.000-125.000,6.282282e+12,Rear Wheel Drive (RWD),125000000,Bojongloa Kidul
1,2018,>1.000 - 1.500 cc,Silver,Hatchback,E CVT,Jakarta D.K.I.,Honda,Automatic,HR-V,"['Airbag', 'Bluetooth Audio', 'Electric Window...",Bursa Mobil Blok M Square,Bensin,Diler,Jakarta Selatan,Indonesia,10.000-15.000,6.289655e+12,4X2,240000000,Kebayoran Baru
2,2017,>1.000 - 1.500 cc,Putih,Hatchback,Panca T,Sumatra Selatan,Datsun,Manual,Go,"['Airbag', 'Electric Window', 'Sensor Parkir']",NaN,Bensin,Diler,Palembang Kota,Indonesia,15.000-20.000,6.281279e+12,Front Wheel Drive (FWD),80000000,Kemuning
3,2013,>1.000 - 1.500 cc,Silver,NaN,E,Jawa Timur,Honda,Automatic,Freed,NaN,NaN,Bensin,NaN,Surabaya Kota,Indonesia,85.000-90.000,6.283832e+12,NaN,170000000,Gubeng
4,2016,>1.000 - 1.500 cc,Putih,MPV,GL Sporty,Jakarta D.K.I.,Suzuki,Manual,Ertiga,"['Airbag', 'Electric Window', 'Sensor Parkir']",Bursa Mobil MGK Kemayoran,Bensin,Diler,Jakarta Pusat,Indonesia,60.000-65.000,6.281912e+12,4X2,125000000,Kemayoran


In [5]:
display(df.shape)
display(df.describe())
display(df.info())

(36575, 20)

,phone,price
count,3.404300e+04,3.657500e+04
mean,5.926282e+12,1.712408e+08
std,5.689054e+12,5.410271e+08
min,6.281110e+10,1.234500e+04
25%,6.281268e+12,8.500000e+07
50%,6.281703e+12,1.200000e+08
75%,6.285215e+12,1.747250e+08
max,6.289581e+13,8.975000e+10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36575 entries, 0 to 36574
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Tahun             36575 non-null  object 
 1   Kapasitas mesin   32192 non-null  object 
 2   Warna             36575 non-null  object 
 3   Tipe bodi         30986 non-null  object 
 4   Varian            34989 non-null  object 
 5   STATE             36575 non-null  object 
 6   Merek             36575 non-null  object 
 7   Transmisi         36575 non-null  object 
 8   Model             36575 non-null  object 
 9   Fitur tambahan    24915 non-null  object 
 10  Nama Bursa Mobil  4959 non-null   object 
 11  Tipe bahan bakar  36575 non-null  object 
 12  Tipe Penjual      30141 non-null  object 
 13  CITY              36575 non-null  object 
 14  COUNTRY           36575 non-null  object 
 15  Jarak tempuh      36575 non-null  object 
 16  phone             34043 non-null  float6

None

## Pre-processing

Remove rows with missing values

In [10]:
display(df.isna().sum())

Tahun                   0
Kapasitas mesin      4383
Warna                   0
Tipe bodi            5589
Varian               1586
STATE                   0
Merek                   0
Transmisi               0
Model                   0
Fitur tambahan      11660
Nama Bursa Mobil    31616
Tipe bahan bakar        0
Tipe Penjual         6434
CITY                    0
COUNTRY                 0
Jarak tempuh            0
phone                2532
Sistem Penggerak     7793
price                   0
NEIGHBOURHOOD           2
dtype: int64

In [11]:
df_ = df.dropna().copy()
display(df_.isna().sum())

Tahun               0
Kapasitas mesin     0
Warna               0
Tipe bodi           0
Varian              0
STATE               0
Merek               0
Transmisi           0
Model               0
Fitur tambahan      0
Nama Bursa Mobil    0
Tipe bahan bakar    0
Tipe Penjual        0
CITY                0
COUNTRY             0
Jarak tempuh        0
phone               0
Sistem Penggerak    0
price               0
NEIGHBOURHOOD       0
dtype: int64

In [12]:
display(df_.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3150 entries, 1 to 36564
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Tahun             3150 non-null   object 
 1   Kapasitas mesin   3150 non-null   object 
 2   Warna             3150 non-null   object 
 3   Tipe bodi         3150 non-null   object 
 4   Varian            3150 non-null   object 
 5   STATE             3150 non-null   object 
 6   Merek             3150 non-null   object 
 7   Transmisi         3150 non-null   object 
 8   Model             3150 non-null   object 
 9   Fitur tambahan    3150 non-null   object 
 10  Nama Bursa Mobil  3150 non-null   object 
 11  Tipe bahan bakar  3150 non-null   object 
 12  Tipe Penjual      3150 non-null   object 
 13  CITY              3150 non-null   object 
 14  COUNTRY           3150 non-null   object 
 15  Jarak tempuh      3150 non-null   object 
 16  phone             3150 non-null   float64

None

Drop duplicates:

In [13]:
df_2 = df_.drop_duplicates(keep="first", inplace=False)
display(df_2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3147 entries, 1 to 36564
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Tahun             3147 non-null   object 
 1   Kapasitas mesin   3147 non-null   object 
 2   Warna             3147 non-null   object 
 3   Tipe bodi         3147 non-null   object 
 4   Varian            3147 non-null   object 
 5   STATE             3147 non-null   object 
 6   Merek             3147 non-null   object 
 7   Transmisi         3147 non-null   object 
 8   Model             3147 non-null   object 
 9   Fitur tambahan    3147 non-null   object 
 10  Nama Bursa Mobil  3147 non-null   object 
 11  Tipe bahan bakar  3147 non-null   object 
 12  Tipe Penjual      3147 non-null   object 
 13  CITY              3147 non-null   object 
 14  COUNTRY           3147 non-null   object 
 15  Jarak tempuh      3147 non-null   object 
 16  phone             3147 non-null   float64

None

Check Variance of every features

In [20]:
for column_name in df_2:
    print('-------',column_name,'-------')
    print(df_2[column_name].value_counts())

------- Tahun -------
2015     360
2013     346
2014     341
2016     341
2017     279
2012     277
2018     193
2011     165
2010     144
2008     108
2009      87
2007      61
2005      58
2006      46
2004      44
2019      41
2003      36
2000      29
2002      27
2001      22
1997      18
1993      18
1994      16
1995      15
<1986     14
1996      14
1990       7
1998       7
1989       6
1999       6
1992       5
1991       5
1987       4
1986       4
1988       3
Name: Tahun, dtype: int64
------- Kapasitas mesin -------
>1.000 - 1.500 cc    1400
>2.000 - 3.000 cc     806
>1.500 - 2.000 cc     713
>3.000 cc             131
<1.000 cc              97
Name: Kapasitas mesin, dtype: int64
------- Warna -------
Hitam      956
Putih      792
Silver     541
Abu-abu    362
Merah      189
Biru        95
Coklat      53
Hijau       41
Marun       30
Kuning      27
Oranye      27
Lainnya     16
Emas        10
Ungu         8
Name: Warna, dtype: int64
------- Tipe bodi -------
MPV            

In [27]:
for column_name in df_2:
    print('-------',column_name,'-------')
    uniques = df_2[column_name].unique()
    print('Uniques:',len(uniques))
    print('uniques/len:', len(uniques)/len(df_2[column_name]))
    print(uniques[:5])

------- Tahun -------
Uniques: 35
uniques/len: 0.011121703209405783
['2018' '2016' '2017' '2012' '2013']
------- Kapasitas mesin -------
Uniques: 5
uniques/len: 0.0015888147442008262
['>1.000 - 1.500 cc' '<1.000 cc' '>1.500 - 2.000 cc' '>3.000 cc'
 '>2.000 - 3.000 cc']
------- Warna -------
Uniques: 14
uniques/len: 0.0044486812837623135
['Silver' 'Putih' 'Hitam' 'Abu-abu' 'Merah']
------- Tipe bodi -------
Uniques: 18
uniques/len: 0.005719733079122974
['Hatchback' 'MPV' 'Minibus' 'Compact & City Car' 'Jeep']
------- Varian -------
Uniques: 405
uniques/len: 0.12869399428026693
['E CVT' 'GL Sporty' 'E' 'Veloz' 'M']
------- STATE -------
Uniques: 25
uniques/len: 0.00794407372100413
['Jakarta D.K.I.' 'Jawa Timur' 'Jawa Barat' 'Sulawesi Selatan'
 'Bangka Belitung']
------- Merek -------
Uniques: 42
uniques/len: 0.01334604385128694
['Honda' 'Suzuki' 'Toyota' 'Daihatsu' 'Land Rover']
------- Transmisi -------
Uniques: 3
uniques/len: 0.0009532888465204957
['Automatic' 'Manual' 'Automatic Tript

Drop columns with low uniques count

In [26]:
df_2.drop(['COUNTRY'], axis='columns', inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Categorical to Numerical Conversion

In [35]:
from sklearn.preprocessing import LabelEncoder

categ_cols = ['Tahun','Kapasitas mesin', 'Warna', 'Tipe bodi', 'Varian', 
          'STATE', 'Merek', 'Transmisi', 'Model', 'Fitur tambahan',
         'Nama Bursa Mobil', 'Tipe bahan bakar', 'Tipe Penjual', 'CITY',
          'Jarak tempuh', 'Sistem Penggerak', 'NEIGHBOURHOOD']

# Label Encoding non-numerical feature
df_encoded = df_2.copy()
for col_name in categ_cols:
    le = LabelEncoder()
    le.fit(df_2[col_name])
    df_encoded[col_name] = le.transform(df_2[col_name]) 
    
df_encoded.head()

,Tahun,Kapasitas mesin,Warna,Tipe bodi,Varian,STATE,Merek,Transmisi,Model,Fitur tambahan,Nama Bursa Mobil,Tipe bahan bakar,Tipe Penjual,CITY,Jarak tempuh,phone,Sistem Penggerak,price,NEIGHBOURHOOD
1,32,1,12,6,170,4,13,0,131,84,4,0,0,55,1,6.289655e+12,0,240000000,197
4,30,1,11,8,209,4,36,2,82,189,9,0,0,54,47,6.281912e+12,0,125000000,209
13,31,1,11,8,169,4,13,0,31,177,14,0,0,57,23,6.285693e+12,0,177000000,319
18,32,1,5,9,376,8,38,0,25,120,5,0,0,135,23,6.281358e+12,3,175000000,507
25,32,0,0,2,253,8,6,2,29,611,15,0,1,135,0,6.285786e+12,0,93500000,152


Check feature correlation

In [36]:
# data_train correlation
corr = df_encoded.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,Tahun,Kapasitas mesin,Warna,Tipe bodi,Varian,STATE,Merek,Transmisi,Model,Fitur tambahan,Nama Bursa Mobil,Tipe bahan bakar,Tipe Penjual,CITY,Jarak tempuh,phone,Sistem Penggerak,price,NEIGHBOURHOOD
Tahun,1.00,-0.09,0.15,-0.13,0.12,-0.23,-0.01,-0.23,-0.01,-0.25,-0.01,0.02,-0.39,0.01,0.13,-0.01,0.11,0.19,0.04
Kapasitas mesin,-0.09,1.00,-0.04,0.30,-0.19,-0.11,0.07,-0.18,0.04,-0.06,0.09,0.43,-0.12,-0.09,-0.03,-0.06,0.00,0.38,0.01
Warna,0.15,-0.04,1.00,-0.07,0.05,-0.03,0.04,-0.05,-0.01,-0.00,-0.01,-0.02,-0.08,0.02,0.07,-0.01,0.01,0.03,0.00
Tipe bodi,-0.13,0.30,-0.07,1.00,-0.18,-0.00,-0.03,-0.03,-0.00,-0.03,0.03,0.01,-0.01,-0.08,-0.03,-0.01,-0.04,0.09,-0.01
Varian,0.12,-0.19,0.05,-0.18,1.00,0.02,0.10,0.04,0.15,-0.01,-0.03,0.02,-0.03,0.04,-0.00,0.03,0.02,-0.06,0.05
STATE,-0.23,-0.11,-0.03,-0.00,0.02,1.00,-0.00,0.26,-0.02,0.16,0.06,-0.00,0.43,0.08,-0.08,0.01,-0.06,-0.12,0.02
Merek,-0.01,0.07,0.04,-0.03,0.10,-0.00,1.00,-0.02,-0.11,0.03,0.00,0.15,0.00,0.01,0.01,0.03,-0.06,-0.05,-0.01
Transmisi,-0.23,-0.18,-0.05,-0.03,0.04,0.26,-0.02,1.00,0.03,0.16,-0.03,-0.00,0.32,0.04,-0.07,0.06,-0.01,-0.10,-0.06
Model,-0.01,0.04,-0.01,-0.00,0.15,-0.02,-0.11,0.03,1.00,0.01,-0.01,0.07,-0.02,0.00,-0.04,0.00,0.01,0.03,0.02
Fitur tambahan,-0.25,-0.06,-0.00,-0.03,-0.01,0.16,0.03,0.16,0.01,1.00,0.05,-0.02,0.32,0.01,-0.08,-0.02,-0.03,-0.05,0.01


Check which features influence the most for price

In [37]:
corr.loc[:,'price'].abs().sort_values(ascending=False)[1:]

Kapasitas mesin     0.383317
Tahun               0.194885
STATE               0.124650
Tipe Penjual        0.113746
Sistem Penggerak    0.106091
Transmisi           0.098221
Tipe bodi           0.092939
Nama Bursa Mobil    0.073205
Tipe bahan bakar    0.062427
Jarak tempuh        0.058734
Varian              0.055676
Merek               0.053979
Fitur tambahan      0.052503
CITY                0.047480
phone               0.040063
Model               0.034885
Warna               0.026872
NEIGHBOURHOOD       0.018271
Name: price, dtype: float64

In [39]:
df_final = df_encoded.copy()

## Modelling

In [41]:
y = df_final['price']
X = df_final.drop(['price'], axis='columns', inplace=False)

In [42]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, Lasso, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score, train_test_split

def cv_rmse(model, x, y):
    r = np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = 5))
    return r

# Percent of the X array to use as training set. This implies that the rest will be test set
test_size = .33

#Split into train and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state = 3)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

r = range(2003, 2017)
km_year = 10000

(2108, 18) (1039, 18) (2108,) (1039,)


In [43]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor()

param_grid = { "criterion" : ["mse"]
              , "min_samples_leaf" : [3]
              , "min_samples_split" : [3]
              , "max_depth": [10]
              , "n_estimators": [500]}

gs = GridSearchCV(estimator=rf, param_grid=param_grid, cv=2, n_jobs=-1, verbose=1)
gs = gs.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    3.9s finished


## Inference

In [44]:
print(gs.best_score_)
print(gs.best_params_)

0.4105460492269854
{'criterion': 'mse', 'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 500}


In [45]:
bp = gs.best_params_
forest = RandomForestRegressor(criterion=bp['criterion'],
                              min_samples_leaf=bp['min_samples_leaf'],
                              min_samples_split=bp['min_samples_split'],
                              max_depth=bp['max_depth'],
                              n_estimators=bp['n_estimators'])
forest.fit(X_train, y_train)
# Explained variance score: 1 is perfect prediction
print('Score: %.2f' % forest.score(X_val, y_val))

Score: 0.53


## Feature Importance Rank

In [52]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")

columns = X_train.columns.values

for f in range(X.shape[1]):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], columns[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center",tick_label = X_train.columns.values)
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

Feature ranking:


NameError: name 'Ximportances' is not defined